#Progetto ESM
#Riconoscimento di volti
Gruppo 7

Simeoni Ildebrando

Scanu Mario

Marescalco Christian

In [ ]:
%reset -f
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import keras
import pandas as pd
import tensorflow as tf

##Download dei dati

In [ ]:
%cd /content/drive/MyDrive/esm/immagini
!pwd
!unzip img_align_celeba.zip  -d /content/drive/MyDrive/esm/immagini/img_align_celeba

##Creazione file csv

In [ ]:
data = np.genfromtxt("/content/drive/MyDrive/esm/identity_CelebA.txt",delimiter=" ").T  
'''genfromtxt ritorna un ndarray (array di dimensione N); .T ne fa la trasposta,
in modo da avere una matrice 2x202599, avente sulla prima riga tutti i nomi 
delle immagini e sulla seconda le relative identità'''

###Calcolo delle 1000 identità più presenti nel dataset

In [ ]:
data = np.delete(data,(0),axis=0) #eliminiamo la prima riga formata dai nomi delle immagini
const = data.shape[1] #numero delle identità (ripetute), pari al numero di immagini presenti nel dataset (202599)
data = np.reshape(data,(const)) #in questo caso reshape da come risultato un array monodimensionale di lunghezza const, che non più un array 1x202599 ma un vettore
identita_counter = {} #conteggio delle volte in cui un'identità si ripete nelle annotazioni del dataset, è un dizionario perché poi facciamo il sort, in un dizionario python non ci sono valori duplicati, quindi sovrascriviamo elementi del dizionario (aventi stessa chiave (identita)) quando si ripetono, aventi stessa chiave, ma nuovo valore pari a quello precedente +1
for identita in data:           #scorro l'array data di 202599 valori
  if identita in identita_counter:  #incremento di uno del counter di una specifica identità ogni volta che si ripresenta nel dataset
     identita_counter[identita] += 1 
  else:
    identita_counter[identita] = 1

popular_identita = sorted(identita_counter, key = identita_counter.get, reverse = True) #effettuiamo il sorting decrescente (reverse=True) del dizionario sulla chiave 'identita_counter' che serve per confrontare gli elementi in base al loro numero di occorrenze nel dizionario 
top_1000 = popular_identita[:1000]  #vettore delle 1000 identità più ricorrenti nel dataset, i valori sono float perché identita_counter era float
top_1000 = [int(i) for i in top_1000] #effettuo una conversione da float a interi

###Suddividisione delle immagini per ogni identità in immagini di training, validazione e test

In [ ]:
items = []    #lista di tutte le tuple,separate da parentesi tonde, del file identity celebA , in cui ogni tupla è formata da nome dell'immagine e relativa identità
file = open('/content/identity_CelebA.txt','r') 
for line in file:
    x, y = line.split() #divido ogni linea del file, che era una stringa, e ho in output due stringhe, la prima nome dell'immagine la seconda identita relativa;con separatore di default la stringa stessa, quindi divide in base allo spazio
    items.append((x, int(y))) #appendo alla lista items, tuple formate da nome immagine e relativa identità (passata come un intero)

img_per_identita = [] #lista di liste in cui ogni lista interna include tutte le tuple associate a una stessa identita
img_per_training = []   # le seguenti sono liste di tuple
img_per_validation = []
img_per_test = []

#inizio for
for identita in top_1000:   #itero solo sulle 1000 identità più presenti nel dataset
  img_identita = []  # singola sottolista della lista img_per_identita, formata da tuple associate a una stessa identita
  for item in items:    #itero su tutte le tuple del file
    if item[1]==identita:    #item[1] è il secondo elemento della singola tupla item appartenente alla lista di tuple items; rappresenta l'identita di quella particolare immagine, qui verifico se l'identità presa in riferimento è uguale a una del file che sto scorrendo 
      img_identita.append((item[0],identita))  #se il secondo elemento della tupla è una delle 1000 identita più presenti, allora aggiungo la tupla alla lista di tuple img_class
  img_per_identita.append(img_identita) #append della sottolista img_identita alla lista di liste img_per_identita
  
  n = identita_counter[identita] #numero totale di immagini per la specifica identita su cui sto iterando presenti nel dataset, dizionario con key identita, che restituisce il numero di elementi del dizionario aventi quella key
  n70 = int(70*n/100)   #suddivisione delle immagini di ognuna delle 1000 identita in 70/20/10
  n20 = int(20*n/100)
  n10 = int(10*n/100)
  nresto = n-n70-n20-n10
  n70 = n70+nresto #approssimazione per eccesso della percentuale di training

  img_70 = img_identita[0:n70]
  img_20 = img_identita[n70:n70+n20]
  img_10 = img_identita[n70+n20:n70+n20+n10]

  for img in img_70:      #per appendere le singole tuple del vettore img_70 una alla volta alla lista img_per_training (che è una lista di tuple)
   img_per_training.append(img)
  for img in img_20:
   img_per_test.append(img)
  for img in img_10:
   img_per_validation.append(img)
#fine for

###Scrittura file csv

In [ ]:
import csv
with open('/content/training.csv', 'w') as csvfile: #vedi documentazione cvwriter
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(img_per_training)
    
with open('/content/test.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(img_per_test)

with open('/content/validation.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(img_per_validation)

##Preparazione dei dati

In [ ]:
img_rows = 224
img_cols =224
img_channels = 3

from keras.preprocessing.image import ImageDataGenerator

traindf=pd.read_csv('/content/drive/MyDrive/esm/csv/training.csv',dtype={'A':str,'B':str},names=['A','B']) #Read a comma-separated values (csv) file into DataFrame.
train_datagen = ImageDataGenerator(rescale = 1./255,rotation_range=20,zoom_range =[0.9,1.2],width_shift_range=0.1,height_shift_range=0.1,preprocessing_function=tf.keras.applications.resnet.preprocess_input)

train_generator = train_datagen.flow_from_dataframe(dataframe = traindf,
                                                    directory='/content/drive/MyDrive/esm/immagini/img_align_celeba',
                                                    target_size = (img_rows,img_cols),
                                                    x_col='A',
                                                    y_col='B',
                                                    class_mode = 'categorical',
                                                    batch_size = 32
                                                    ) 


validationdf=pd.read_csv('/content/drive/MyDrive/esm/csv/validation.csv',dtype={'A':str,'B':str},names=['A','B']) 
validation_datagen = ImageDataGenerator(rescale = 1./255,preprocessing_function=tf.keras.applications.resnet.preprocess_input)

validation_generator = validation_datagen.flow_from_dataframe(dataframe = validationdf,
                                                    directory='/content/drive/MyDrive/esm/immagini/img_align_celeba',
                                                    target_size = (img_rows,img_cols),
                                                    x_col='A',
                                                    y_col='B',
                                                    class_mode = 'categorical',
                                                    batch_size = 32
                                                    ) 

testdf=pd.read_csv('/content/drive/MyDrive/esm/csv/test.csv',dtype={'A':str,'B':str},names=['A','B']) 
test_datagen = ImageDataGenerator(rescale = 1./255,preprocessing_function=tf.keras.applications.resnet.preprocess_input)

test_generator = test_datagen.flow_from_dataframe(dataframe = testdf,
                                                    directory='/content/drive/MyDrive/esm/immagini/img_align_celeba',
                                                    target_size = (img_rows,img_cols),
                                                    x_col='A',
                                                    y_col='B',
                                                    class_mode = 'categorical',
                                                    shuffle=False,
                                                    batch_size = 32
                                                    ) 


##Architettura
ResNet50

In [ ]:
model=tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation = "softmax"
)

opt= tf.keras.optimizers.Adam(learning_rate=0.0003, name="Adam") 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

##Addestramento

In [ ]:
num_imm_train = len(train_generator.filenames)
num_imm_val = len(validation_generator.filenames)
batch_size = 32
num_epoche = 15 

model.fit(
    x=train_generator,
    y=None,
    batch_size=batch_size,
    epochs=num_epoche,
    verbose=1,
    validation_data = validation_generator,
    steps_per_epoch=num_imm_train // batch_size,
    validation_steps=num_imm_val//batch_size,
    validation_batch_size=batch_size,
    workers=10,
    use_multiprocessing=True,
)


##Valutazione delle prestazioni

###Valutazioni sull'intero test set

In [ ]:
pred = model.predict(test_generator) 
label = test_generator.classes  #classes variabile dell'oggetto test_generator,(rapp.sparsa),label è etichetta effettiva
pred1=np.argmax(pred,1) #rapp sparsa,etichetta predetta

import sklearn.metrics as metrics
checonfusione = metrics.confusion_matrix(label,pred1)

print(metrics.accuracy_score(label,pred1))
print(metrics.confusion_matrix(label,pred1))

plt.figure(1)
plt.imshow(checonfusione,clim = None,cmap = 'jet')



###Valutazioni su singole immagini

In [ ]:
testsingole_datagen = ImageDataGenerator(rescale = 1./255,preprocessing_function=tf.keras.applications.resnet.preprocess_input)
test_generatorsingole = test2_datagen.flow_from_directory('/content/testsingole',
                                                   target_size=(img_rows, img_cols),
                                                   batch_size=1,
                                                   class_mode='categorical')                                                                                                        

pred = model.predict(test_generatorsingole)   
pred1=np.argmax(pred,1)  
print(pred1)

#Top 3 prediction

Top3 = np.sort(pred[0])[::-1]
    
print(pred[0].argsort()[-3:][::-1]) #Top 3 delle classi più probabili
print(Top3[0:3])                    #Relative probabilità